# Feature Engineering and Polynomials 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, LassoCV, RidgeCV

In [2]:
df = pd.read_pickle('grouped_act_net.pkl')

In [3]:
df.head()

,year,rating,num_reviews,genre,certificate,num_episodes,actors,network,runtime,awards,top_actors,top_networks
title,,,,,,,,,,,,
Level Up,2012.0,4.3,897,Sci-Fi,TV-PG,36,Gaelan Connell,Cartoon Network,30.0,0.0,other,other
Behind the Scenes,2012.0,8.0,12,Comedy,Not Rated,68,Keith Stoller,Lost Angeles Films,90.0,0.0,other,other
Mama Drama,2012.0,5.9,28,Reality-TV,Not Rated,10,Missing,495 Productions,60.0,0.0,Missing,other
Oprah Prime,2012.0,4.5,212,Talk-Show,Not Rated,86,Oprah Winfrey,Harpo Productions,60.0,1.0,other,other
Gator Boys,2012.0,7.0,609,Reality-TV,Not Rated,36,Paul Bedard,Mike Mathis Productions,60.0,0.0,other,other


In [4]:
df.drop(columns=['actors', 'network'], inplace=True)

In [5]:
df2 = pd.get_dummies(df, drop_first=True)

In [6]:
df2.head()

,year,rating,num_reviews,num_episodes,runtime,awards,genre_Adventure,genre_Animation,genre_Biography,genre_Comedy,...,top_networks_Embassy Row,top_networks_Entertainment One,top_networks_Leftfield Pictures,top_networks_Missing,top_networks_Netflix,top_networks_Pilgrim Studios,top_networks_Sharp Entertainment,top_networks_Vice Media,top_networks_World of Wonder Productions,top_networks_other
title,,,,,,,,,,,,,,,,,,,,,
Level Up,2012.0,4.3,897,36,30.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Behind the Scenes,2012.0,8.0,12,68,90.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
Mama Drama,2012.0,5.9,28,10,60.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Oprah Prime,2012.0,4.5,212,86,60.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Gator Boys,2012.0,7.0,609,36,60.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
X, y = df2.drop('rating',axis=1), df2['rating']
X['num_reviews2'] = X['num_reviews'] ** 2
X['runtime2'] = X['runtime'] ** 2

In [8]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [9]:
# Scale data
std = StandardScaler()
std.fit(X_train.values)

StandardScaler()

In [10]:
## Scale the Predictors on both the train and test set
X_tr = std.transform(X_train.values)
X_te = std.transform(X_test.values)

In [11]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

alphavec = 10**np.linspace(-2,2,200)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_tr, y_train)

LassoCV(alphas=array([1.00000000e-02, 1.04737090e-02, 1.09698580e-02, 1.14895100e-02,
       1.20337784e-02, 1.26038293e-02, 1.32008840e-02, 1.38262217e-02,
       1.44811823e-02, 1.51671689e-02, 1.58856513e-02, 1.66381689e-02,
       1.74263339e-02, 1.82518349e-02, 1.91164408e-02, 2.00220037e-02,
       2.09704640e-02, 2.19638537e-02, 2.30043012e-02, 2.40940356e-02,
       2.52353917e-02, 2.64308149e-0...
       3.44896226e+01, 3.61234270e+01, 3.78346262e+01, 3.96268864e+01,
       4.15040476e+01, 4.34701316e+01, 4.55293507e+01, 4.76861170e+01,
       4.99450512e+01, 5.23109931e+01, 5.47890118e+01, 5.73844165e+01,
       6.01027678e+01, 6.29498899e+01, 6.59318827e+01, 6.90551352e+01,
       7.23263390e+01, 7.57525026e+01, 7.93409667e+01, 8.30994195e+01,
       8.70359136e+01, 9.11588830e+01, 9.54771611e+01, 1.00000000e+02]),
        cv=5)

In [12]:
lasso_model.alpha_

0.010473708979594498

In [13]:
list(zip(X_train.columns, lasso_model.coef_))

[('year', 0.0),
 ('num_reviews', 0.2434826381495916),
 ('num_episodes', -0.014696627406811817),
 ('runtime', -0.028498204512458793),
 ('awards', 0.08601203119667651),
 ('genre_Adventure', 0.013182082450059399),
 ('genre_Animation', -0.06508891817389642),
 ('genre_Biography', 0.03907899545957218),
 ('genre_Comedy', 0.0016834378576660414),
 ('genre_Crime', 0.057749792418045086),
 ('genre_Documentary', 0.02667923644090548),
 ('genre_Drama', 0.03658055554440219),
 ('genre_Family', -0.005490571406921406),
 ('genre_Fantasy', -0.0),
 ('genre_Game-Show', -0.12753627805210113),
 ('genre_History', 0.02605551449694859),
 ('genre_Horror', -0.005969823291463971),
 ('genre_Missing', 0.025760815216558354),
 ('genre_Music', 0.014631218020731706),
 ('genre_Musical', -0.02641895924817603),
 ('genre_Mystery', 0.0),
 ('genre_News', -0.09791663973845204),
 ('genre_Reality-TV', -0.15557395744177613),
 ('genre_Romance', 0.0),
 ('genre_Sci-Fi', -0.04632917521388038),
 ('genre_Short', 0.05102359105819083),
 ('

In [14]:
# Make predictions on the test set using the new model
test_set_pred = lasso_model.predict(X_te)

In [16]:
#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

In [17]:
# Find the MAE and R^2 on the test set using this model
mae(y_test, test_set_pred)

1.0332144978000777

In [19]:
from sklearn.metrics import r2_score
r2_score(y_test, test_set_pred)

0.08781830632641208

In [4]:
df.head()

,title,year,rating,num_reviews,genre,certificate,num_episodes,actors,network,runtime,awards,top_actors
0,Level Up,2012.0,4.3,897,Sci-Fi,TV-PG,36,Gaelan Connell,Cartoon Network,30.0,0.0,other
1,Behind the Scenes,2012.0,8.0,12,Comedy,Not Rated,68,Keith Stoller,Lost Angeles Films,90.0,0.0,other
2,Mama Drama,2012.0,5.9,28,Reality-TV,Not Rated,10,Missing,495 Productions,60.0,0.0,Missing
3,Oprah Prime,2012.0,4.5,212,Talk-Show,Not Rated,86,Oprah Winfrey,Harpo Productions,60.0,1.0,Oprah Winfrey
4,Gator Boys,2012.0,7.0,609,Reality-TV,Not Rated,36,Paul Bedard,Mike Mathis Productions,60.0,0.0,other


In [5]:
df.top_actors.value_counts()

other             4053
Missing            353
Aaron Elliott        8
Gordon Ramsay        6
Steve Harvey         6
Chris Hardwick       6
Kal Penn             6
John Beach           5
Will Arnett          5
Keven Santos         5
Ashleigh Ball        5
Eric Whiteley        5
Nicole Byer          4
Duff Goldman         4
Hank Azaria          4
Guy Fieri            4
Joel McHale          4
Ben Sinclair         4
Oprah Winfrey        4
Tim Heidecker        4
Eric Bauza           4
Name: top_actors, dtype: int64

In [8]:
# group networks

df.network.value_counts()

Missing                                 654
Amazon Studios                           48
ABC Signature                            23
DreamWorks Animation Television          22
3 Arts Entertainment                     22
                                       ... 
Mr. Big Shot Fancy-Pants Productions      1
Ideas In Intertainment                    1
Tandem Communications                     1
Talpa Media USA                           1
Wanda Wayne Productions                   1
Name: network, Length: 2281, dtype: int64

In [6]:
# choose only top 20 networks
top_networks = list(df.network.value_counts().head(20).keys())
def big_network(string):
    if string in (top_networks):
        return string
    else:
        return 'other'
df['top_networks'] = df.network.apply(big_network)

In [7]:
df.top_networks.value_counts()

other                              3510
Missing                             654
Amazon Studios                       48
ABC Signature                        23
DreamWorks Animation Television      22
3 Arts Entertainment                 22
Netflix                              21
Berlanti Productions                 20
Disney Television Animation          19
Blind Wave                           16
Bunim-Murray Productions (BMP)       16
Entertainment One                    15
Cartoon Network Studios              15
Sharp Entertainment                  14
Vice Media                           13
Leftfield Pictures                   12
Bellum Entertainment                 12
Embassy Row                          12
Pilgrim Studios                      12
CBS Television Studios               12
Broadway Video                       11
Name: top_networks, dtype: int64

In [8]:
df.set_index('title', inplace=True)
df.head()

,year,rating,num_reviews,genre,certificate,num_episodes,actors,network,runtime,awards,top_actors,top_networks
title,,,,,,,,,,,,
Level Up,2012.0,4.3,897,Sci-Fi,TV-PG,36,Gaelan Connell,Cartoon Network,30.0,0.0,other,other
Behind the Scenes,2012.0,8.0,12,Comedy,Not Rated,68,Keith Stoller,Lost Angeles Films,90.0,0.0,other,other
Mama Drama,2012.0,5.9,28,Reality-TV,Not Rated,10,Missing,495 Productions,60.0,0.0,Missing,other
Oprah Prime,2012.0,4.5,212,Talk-Show,Not Rated,86,Oprah Winfrey,Harpo Productions,60.0,1.0,Oprah Winfrey,other
Gator Boys,2012.0,7.0,609,Reality-TV,Not Rated,36,Paul Bedard,Mike Mathis Productions,60.0,0.0,other,other


In [9]:

df2 = df.copy(deep=True)

Baseline Model R^2 is 0.026. Looking at p-values, it seems like only awards and num_reviews are significant.

In [10]:
new_df = pd.get_dummies(df2, drop_first=True)

In [11]:
new_df.head()

,year,rating,num_reviews,num_episodes,runtime,awards,genre_Adventure,genre_Animation,genre_Biography,genre_Comedy,...,top_networks_DreamWorks Animation Television,top_networks_Embassy Row,top_networks_Entertainment One,top_networks_Leftfield Pictures,top_networks_Missing,top_networks_Netflix,top_networks_Pilgrim Studios,top_networks_Sharp Entertainment,top_networks_Vice Media,top_networks_other
title,,,,,,,,,,,,,,,,,,,,,
Level Up,2012.0,4.3,897,36,30.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Behind the Scenes,2012.0,8.0,12,68,90.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
Mama Drama,2012.0,5.9,28,10,60.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Oprah Prime,2012.0,4.5,212,86,60.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Gator Boys,2012.0,7.0,609,36,60.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
# Split data into train and test sets

X, y = new_df.drop('rating',axis=1), new_df['rating']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=42) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

array([8.3, 7.5, 5.4, ..., 6.5, 9.3, 8.5])

In [13]:
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

Simple regression scores:  [-1538614912894.728, -8457916615805.401, -22812228642010.605, -146297147929.97153, -314568424698.0218]
Ridge scores:  [-0.843244578956601, -0.7717701490136091, -0.7759487363756521, -0.546057263469246, -0.4956293334809858] 

Simple mean cv r^2: -6653925148667.746 +- 8637687441374.713
Ridge mean cv r^2: -0.687 +- 0.139


In [12]:
# add polynomials

new_df['num_reviews2'] = new_df['num_reviews'] ** 2
new_df['runtime2'] = new_df['runtime'] ** 2

In [11]:
new_df.head()

,year,rating,num_reviews,num_episodes,runtime,awards,title_#ActorsLife,title_#Adulting,title_#NoFilter with Ana Kasparian,title_*Loosely Exactly Nicole,...,top_networks_Embassy Row,top_networks_Entertainment One,top_networks_Leftfield Pictures,top_networks_Missing,top_networks_Netflix,top_networks_Pilgrim Studios,top_networks_Sharp Entertainment,top_networks_Vice Media,top_networks_World of Wonder Productions,top_networks_other
0,2012.0,4.3,897,36,30.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2012.0,8.0,12,68,90.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2012.0,5.9,28,10,60.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2012.0,4.5,212,86,60.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2012.0,7.0,609,36,60.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
# Split data into train and test sets

X, y = new_df.drop('rating',axis=1), new_df['rating']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=42) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

Simple regression scores:  [-11.119207346768032, -142.65240458261263, -10707.790081552128, -114.85421822758593, -1242.992937136722]
Ridge scores:  [-0.2334530641158976, -0.04284306783890446, -0.29111317066040954, -0.4541650539526556, -0.16786656081090023] 

Simple mean cv r^2: -2443.882 +- 4156.265
Ridge mean cv r^2: -0.238 +- 0.136


In [ ]:
R^2 value decreased with polynomial terms to 0.058